In [5]:
import numpy as np
import pandas as pd
import warnings


warnings.filterwarnings('ignore')

---

In [5]:
# !pip install python-binance

In [9]:
# IMPORTS
import pandas as pd
import math
import os.path
import time
# from bitmex import bitmex
from binance import Client
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

### API
# bitmex_api_key = '[REDACTED]'    #Enter your own API-key here
# bitmex_api_secret = '[REDACTED]' #Enter your own API-secret here
binance_api_key = 'NFOGKHaUD9vqYOJ0HHFnmOBPhiPevnYOcrORt1ByoghEx32VSD3Qmlp1iy4Etzka'    #Enter your own API-key here
binance_api_secret = '7GZOgQrmvA9BmD1aUq5yTho4AA4ccONCnxyRMJ2H9pgPQQvF6bQiUrL3gy7QUDXN' #Enter your own API-secret here

### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
# bitmex_client = bitmex(test=False, api_key=bitmex_api_key, api_secret=bitmex_api_secret)
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)


### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

# def get_all_bitmex(symbol, kline_size, save = False):
#     filename = '%s-%s-data.csv' % (symbol, kline_size)
#     if os.path.isfile(filename): data_df = pd.read_csv(filename)
#     else: data_df = pd.DataFrame()
#     oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
#     delta_min = (newest_point - oldest_point).total_seconds()/60
#     available_data = math.ceil(delta_min/binsizes[kline_size])
#     rounds = math.ceil(available_data / batch_size)
#     if rounds > 0:
#         print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
#         for round_num in tqdm_notebook(range(rounds)):
#             time.sleep(1)
#             new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
#             data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
#             temp_df = pd.DataFrame(data)
#             data_df = data_df.append(temp_df)
#     data_df.set_index('timestamp', inplace=True)
#     if save and rounds > 0: data_df.to_csv(filename)
#     print('All caught up..!')
#     return data_df

In [21]:
df = get_all_binance("BTCUSDT", "1m", save=True)

All caught up..!


In [22]:
df

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2017-08-17 04:00:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,1.77518300,1502942459999,7564.90685084,3,0.07518300,320.39085084,7960.54017996
2017-08-17 04:01:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.00000000,1502942519999,0.00000000,0,0.00000000,0.00000000,7960.45262989
2017-08-17 04:02:00,4280.56000000,4280.56000000,4280.56000000,4280.56000000,0.26107400,1502942579999,1117.54292144,2,0.26107400,1117.54292144,7960.37529526
2017-08-17 04:03:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.01200800,1502942639999,51.17185184,3,0.01200800,51.17185184,7960.37529526
2017-08-17 04:04:00,4261.48000000,4261.48000000,4261.48000000,4261.48000000,0.14079600,1502942699999,599.99933808,1,0.14079600,599.99933808,7960.37529526
...,...,...,...,...,...,...,...,...,...,...,...
2022-06-10 11:07:00,29990.78000000,30013.65000000,29990.78000000,30013.64000000,39.86735000,1654859279999,1196121.01659260,566,27.25875000,817795.98794470,0
2022-06-10 11:08:00,30013.65000000,30022.55000000,30007.24000000,30020.16000000,37.53297000,1654859339999,1126487.20601910,560,20.80941000,624591.70159040,0
2022-06-10 11:09:00,30020.16000000,30020.17000000,30009.06000000,30012.57000000,9.27996000,1654859399999,278523.98856290,340,3.43002000,102934.74497170,0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2523471 entries, 2017-08-17 04:00:00 to 2022-06-10 11:11:00
Data columns (total 11 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   open         object
 1   high         object
 2   low          object
 3   close        object
 4   volume       object
 5   close_time   int64 
 6   quote_av     object
 7   trades       int64 
 8   tb_base_av   object
 9   tb_quote_av  object
 10  ignore       object
dtypes: int64(2), object(9)
memory usage: 231.0+ MB


In [24]:
df.shape

(2523471, 11)

In [25]:
pd.to_numeric(df)

TypeError: arg must be a list, tuple, 1-d array, or Series